In [ ]:
import subprocess, textwrap, operator, base64
from langgraph.graph import END, START, StateGraph
from langgraph.types import Send, interrupt, Command
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from typing import TypedDict
from openai import OpenAI
from typing_extensions import Annotated

memory = InMemorySaver()

llm = init_chat_model("openai:gpt-5-nano")


class State(TypedDict):
    video_file: str
    audio_file: str
    transcription: str
    summaries: Annotated[list[str], operator.add]
    thumbnail_prompts: Annotated[list[str], operator.add]
    thumbnail_sketches: Annotated[list[str], operator.add]
    final_summary: str
    user_feedback: str
    chosen_prompt: str

In [ ]:
def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4", "mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file
    ]
    subprocess.run(command)
    return {
        "audio_file": output_file
    }

def transcribe_audio(state: State):
    client = OpenAI()
    with open(state["audio_file"], "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            # language="en",
            # prompt="to give the model some hints for audio file"
        )
        return {
            "transcription": transcription
        }

def dispatch_summarizers(state: State):
    transcription = state["transcription"]
    chunks = []
    for i, chunk in enumerate(textwrap.wrap(transcription, 500)):
        chunks.append({"id": i + 1, "chunk": chunk})
    return [Send("summarize_chunk", chunk) for chunk in chunks]

def summarize_chunk(chunk_item):
    chunk_id = chunk_item["id"]
    chunk = chunk_item["chunk"]
    # print(f"Summarizing chunk id: {chunk_id} chunk: {chunk[:100]}\n\n====\n\n")

    response = llm.invoke(
        f"""
        Please summarize the following text.

        Text: {chunk}
        """
    )
    summary = f"[Chunk {chunk_id}] {response.content}"
    return {
        "summaries": [summary]
    }

def mega_summary(state: State):
    all_summaries = "\n".join(state["summaries"])
    prompt = f"""
        You are given multiple summaries of different chunks from a video transcription.

        Please create a comprehensive final summary that combines all the key points.

        Individual summaries:

        {all_summaries}
    """
    response = llm.invoke(prompt)

    return {
        "final_summary": response.content
    }

def dispatch_artists(state: State):
    return [
        Send("generate_thumbnails", {"id": i, "summary": state["final_summary"]}) for i in [1, 2, 3]
    ]

def generate_thumbnails(args):
    concept_id = args["id"]
    summary = args["summary"]
    prompt = f"""
        Based on this video summary, create a detailed visual prompt for a YouTube thumbnail.

        Create a detailed prompt for generating a thumbnail image that would attract viewers. Include:
            - Main visual elements
            - Color scheme
            - Text overlay suggestions
            - Overall composition

        Summary: {summary}
    """
    response = llm.invoke(prompt)

    thumbnail_prompt = response.content
    
    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=thumbnail_prompt,
        quality="low",
        moderation="low",
        size="auto"
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)

    filename = f"thumbnail_{concept_id}.jpg"

    with open(filename, "wb") as file:
        file.write(image_bytes)
    
    return {
        "thumbnail_prompts": [thumbnail_prompt],
        "thumbnail_sketches": [filename]
    }

def human_feedback(state: State):
    answer = interrupt({
        "chosen_thumbnail": "Which thumbnail do you like the most?",
        "feedback": "Provide any feedback or changes you'd like for the final thumbnail."
    })
    user_feedback = answer["user_feedback"]
    chosen_prompt = answer["chosen_prompt"]
    return {
        "user_feedback": user_feedback,
        "chosen_prompt": state["thumbnail_prompts"][chosen_prompt-1],
    }

def generate_hd_thumbnail(state: State):
    chosen_prompt = state["chosen_prompt"]
    user_feedback = state["user_feedback"]
    prompt = f"""
        You are a professional YouTube thumbnail designer. Take this original thumbnail prompt and create an enhanced version that incorporates the user's specific feedback.

        ORIGINAL PROMPT:
        {chosen_prompt}

        USER FEEDBACK TO INCORPORATE:
        {user_feedback}

        Create an enhanced prompt that:
            1. maintains the core concept from the original prompt
            2. Specifically addresses and implements the user's feedback request.
            3. Adds professional YouTube thumbnail specifications:
                - high contrast and bold visual elements
                - Clear focal points that draw the eye
                - Professional lighting and composition
                - Optimal text placement and readability with generous padding from edges
                - Colors that pop and grab attention
                - Elements that work well at small thumbnail sizes
                - IMPORTANT: Always ensure adequate white space/padding between any text and the image borders
    """

    response = llm.invoke(prompt)

    final_thumbnail_prompt = response.content
    
    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=final_thumbnail_prompt,
        quality="high",
        moderation="low",
        size="auto"
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)

    filename = f"thumbnail_final.jpg"

    with open(filename, "wb") as file:
        file.write(image_bytes)



In [35]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)
graph_builder.add_node("summarize_chunk", summarize_chunk)
graph_builder.add_node("mega_summary", mega_summary)
graph_builder.add_node("generate_thumbnails", generate_thumbnails)
graph_builder.add_node("human_feedback", human_feedback)
graph_builder.add_node("generate_hd_thumbnail", generate_hd_thumbnail)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_conditional_edges("transcribe_audio", dispatch_summarizers, ["summarize_chunk"])
graph_builder.add_edge("summarize_chunk", "mega_summary")
graph_builder.add_conditional_edges("mega_summary", dispatch_artists, ["generate_thumbnails"])
graph_builder.add_edge("generate_thumbnails", "human_feedback")
graph_builder.add_edge("human_feedback", "generate_hd_thumbnail")
graph_builder.add_edge("generate_hd_thumbnail", END)

graph = graph_builder.compile(checkpointer=memory)

In [36]:
config = {
    "configurable": {
        "thread_id": 1
    },
}

In [37]:
graph.invoke(
    {"video_file": "interview.mp4"},
    config=config
)

{'video_file': 'interview.mp4',
 'audio_file': 'interview.mp3',
 'transcription': '서울 고려대학교 미역하에서 시작한 천 원짜리 영철 버거 핫도그 빵 사이에 고기 볶음과 양배추 소스 등을 넣은 버거는 큰 인기를 끌며 고려대의 명물이 됐습니다. 2000년 영철 버거를 상업한 이영철 씨는 2004년부터 고려대학교에 매년 2천만 원을 기부했습니다. 학생들의 주머니 사정을 생각해 적자에도 천 원의 약속을 지켰습니다. 2015년엔 결국 경영난으로 폐업했는데 고려대 동문들이 나서 가게를 일으켜 세우며 화제가 되기도 했습니다. 10년 전 JTBC와의 인터뷰에서는 재개업을 앞둤던 이 씨의 두려움과 설렘이 고스란히 담겨 있었습니다. 기분이야 뭐 정말 새롭죠. 많이 걱정도 되고 두렵기도 하고 고맙기도 하고 앞으로 성장하고 있으면 어떻게 하실 거예요? 소심으로 돌아가서 열심히 일하고 또 학생들하고 많이 소통하고 하려고 합니다. 이 씨는 인터뷰 내내 미소를 지었습니다. 너무너무 감사하고 정말 그 고마움 잊지 않고 내가 살아가는 모습으로 보답하려고 해요. 암투병 중이던 이 씨는 지난 13일 18살의 나이로 세상을 떠났습니다. 빈손은 이 씨의 평생일도 고려대학교 안암병원 장례식장에 마련됐습니다. 외롭고 방황하던 청춘의 허기진 마음들을 이불처럼 덮어주셨던 사장님의 삶을 생기겠다 웃음이 눈가에 발자국 남는 사람이 되어 기억하겠다. 온라인 후보에는 1,000개 넘는 추모와 추억의 메시지가 속속 담겨지고 있습니다. 이 씨의 발의는 내일 이뤄집니다.\n',
 'summaries': ['[Chunk 1] - 서울 고려대학교에서 큰 인기를 끈 천 원짜리 영철 버거 핫도그는 빵 사이에 고기 볶음과 양배추 소스를 넣은 고유의 메뉴로 명물이 됐다.\n- 이영철 씨가 이 버거를 2000년에 상업화했고, 2004년부터 매년 2천만 원을 고려대에 기부했다. 학생들의 주머니 사정을 고려해 천 원 가격 약속을 지켰다.\n- 2015년 경영난으로 가

In [ ]:
response = {
    "user_feedback": "Make sure the mood of the thumbnail not much sad. and give it a photo realistic, 3d style.",
    "chosen_prompt": 1,
}

('human_feedback',)

In [ ]:
graph.invoke(
    Command(resume=response),
    config=config
)